In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../beauty_data/processed_data/user_test.json'
user_valid_path = '../beauty_data/processed_data/user_valid.json'
user_train_path = '../beauty_data/processed_data/user_train.json'

In [3]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

624


In [4]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

624


In [5]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
print(len(user_train_dict))

624


In [6]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[7, 8, 9, 9, 10]
[7, 8, 9, 9, 10, 11]
[8, 9, 9, 10, 11, 12]


In [7]:
len(user_train_dict[user_id])

5

In [8]:
item_information_path = '../beauty_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

1216


In [9]:
print(item_information_dict['0'])

{'title': 'Fresh Eau De Parfum EDP - Fig Apricot 3.4oz (100ml)', 'brand': 'Fresh', 'price': '', 'description': '\n\nProduct Description: KMRN Fresh Line is a hypoallergenic fragrance, praised for its pleasant scent and ability to generate compliments. However, it may not offer a wide range of scents.'}


In [10]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


In [11]:
# def get_item_description(item_information):
#     item_details = ""
#     if 'title' in item_information:
#         item_details += item_information['title']
#     if 'brand' in item_information:
#         item_details += f" Brand: {item_information['brand']}"
#     if 'price' in item_information:
#         item_details += f" Price: {item_information['price']}"
#     if 'description' in item_information:
#         item_details += f" Description: {item_information['description']}"
#     return item_details

In [11]:
train_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_train_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_items = []
    disliked_items = []

    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            liked_items.append(item)
        else:
            disliked_items.append(item)
    
    input = ""
    if len(liked_items) > 0:
        input += "\nUser Preference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if len(disliked_items) > 0:
        input += "\nUser Unpreference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if str(item_list[-1]) not in item_information_dict:
        continue
    target_item = item_information_dict[str(item_list[-1])]['title']
    input += f"\n Whether the user will like the target movie \"{target_item}\"?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['item'] == item_list[-1]]['rating'].values[0] > 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    train_data.append(user_dict)

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 274 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [12]:
print(len(train_data))

624


In [13]:
train_data[0]

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".',
 'input': '\nUser Preference:"Fresh Eau De Parfum EDP - Fig Apricot 3.4oz (100ml)", "Monoi - Monoi Pitate Jasmine 4 fl oz", "Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz", "Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.", "Bonne Bell Smackers Bath and Body Starburst Collection", \n Whether the user will like the target movie "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz"?',
 'output': 'Yes.'}

In [14]:
total_len = 0
for data in train_data:
    total_len += len(data['input'].split(' '))
print(total_len/len(train_data))

70.01442307692308


In [15]:
# cnt = 0
cleaned_train_data = []
for data in train_data:
    if len(data['input'].split(' ')) <= 120:
        cleaned_train_data.append(data)
print(len(cleaned_train_data))

594


In [16]:
total_len = 0
for data in train_data:
    total_len += len(data['input'])
print(total_len/len(train_data))

436.71153846153845


In [17]:
total_len = 0
for data in cleaned_train_data:
    total_len += len(data['input'])
print(total_len/len(cleaned_train_data))

411.05387205387206


In [18]:
valid_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_valid_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_items = []
    disliked_items = []

    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            liked_items.append(item)
        else:
            disliked_items.append(item)
    
    input = ""
    if len(liked_items) > 0:
        input += "\nUser Preference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if len(disliked_items) > 0:
        input += "\nUser Unpreference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    
    if str(item_list[-1]) not in item_information_dict:
        continue
    target_item = item_information_dict[str(item_list[-1])]['title']
    input += f"\n Whether the user will like the target movie \"{target_item}\"?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['item'] == item_list[-1]]['rating'].values[0] > 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    valid_data.append(user_dict)

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [19]:
total_len = 0
for data in valid_data:
    total_len += len(data['input'].split(' '))
print(total_len/len(valid_data))

81.78846153846153


In [20]:
cleaned_valid_data = []
for data in valid_data:
    if len(data['input'].split(' ')) <= 120:
        cleaned_valid_data.append(data)
print(len(cleaned_valid_data))

576


In [21]:
print(cleaned_train_data[1])
print(cleaned_valid_data[1])

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry", "essie nail polish, cuticle care, primers and finishers", "essie nail polish, cuticle care, primers and finishers", \nUser Unpreference:"Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry", "essie nail polish, cuticle care, primers and finishers", "essie nail polish, cuticle care, primers and finishers", \n Whether the user will like the target movie "Utopia Care Stainless Steel Fingernail Clipper"?', 'output': 'Yes.'}
{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Pre de

In [22]:
test_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_test_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_items = []
    disliked_items = []

    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            liked_items.append(item)
        else:
            disliked_items.append(item)
    
    input = ""
    if len(liked_items) > 0:
        input += "\nUser Preference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if len(disliked_items) > 0:
        input += "\nUser Unpreference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    
    if str(item_list[-1]) not in item_information_dict:
        continue
    target_item = item_information_dict[str(item_list[-1])]['title']
    input += f"\n Whether the user will like the target movie \"{target_item}\"?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['item'] == item_list[-1]]['rating'].values[0] > 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    test_data.append(user_dict)

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [24]:
total_len = 0
for data in test_data:
    total_len += len(data['input'].split(' '))
print(total_len/len(test_data))

91.7315112540193


In [25]:
print(len(test_data))
cleaned_test_data = []
for data in test_data:
    if len(data['input'].split(' ')) <= 120:
        cleaned_test_data.append(data)
print(len(cleaned_test_data))

622
559


In [26]:
print(len(cleaned_train_data))
# print(len(cleaned_valid_data))
print(len(cleaned_test_data))

594
559


In [29]:
print(cleaned_train_data[0])
print(cleaned_valid_data[0])
print(cleaned_test_data[0])

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \nUser Unpreference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \n Whether the user will like the target movie "DJT Cowl Neck Button Embellished Ruched Long Sleeve Blouse Top Grey XL"?', 'output': 'No.'}
{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Allegra K Woman Boat Neck Button Decor Front Batwing 

In [27]:
train_valid_data = cleaned_train_data.copy()
train_valid_data.extend(cleaned_valid_data)
print(len(train_valid_data), train_valid_data[:3])
print(len(train_data), len(valid_data), len(cleaned_test_data))

1170 [{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Fresh Eau De Parfum EDP - Fig Apricot 3.4oz (100ml)", "Monoi - Monoi Pitate Jasmine 4 fl oz", "Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz", "Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.", "Bonne Bell Smackers Bath and Body Starburst Collection", \n Whether the user will like the target movie "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz"?', 'output': 'Yes.'}, {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry", "essie nail polish, cuticle care, primers

### Train Valid split 70:30

In [28]:
import random
random_seed = 42
random.seed(random_seed)
random.shuffle(train_valid_data)

train_valid_split = 0.7
split_index = int(train_valid_split*len(train_valid_data))
print(split_index)

819


In [29]:
new_train_data = train_valid_data[:split_index]
new_valid_data = train_valid_data[split_index:]

print("Train Data: ", len(new_train_data), new_train_data[:3])
print("Valid Data: ", len(new_valid_data), new_valid_data[:3])

Train Data:  819 [{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz", "Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.", "Bonne Bell Smackers Bath and Body Starburst Collection", "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz", \n Whether the user will like the target movie "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz"?', 'output': 'Yes.'}, {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"BeautyMe Blackhead and Pimple Remover Kit with 7 Surgical Extractor Tool", "Beard Trimmer Kit, 5 in 1 Multi-functional Body Groomer Kit of Mustache Trimmer, Nose Hai

### Storing as json

In [30]:
import json

with open('./data/train.json', 'w') as train_file:
    json.dump(new_train_data, train_file)

with open('./data/valid.json', 'w') as valid_file:
    json.dump(new_valid_data, valid_file)

with open('./data/test.json', 'w') as test_file:
    json.dump(cleaned_test_data, test_file)

In [31]:
with open('./data/test.json', 'w') as test_file:
    json.dump(cleaned_test_data, test_file)

In [32]:
print(f"Ground Truth: {test_data[4]}")
content = test_data[4]["instruction"] + test_data[4]["input"]
print(content)
print('*'*100)
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0.1,
  max_tokens=64,
  top_p=0.9
)

response = response.choices[0].message.content
print(response)
len(response.split(' '))

Ground Truth: {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Citre Shine Moisture Burst Shampoo - 16 fl oz", "Bonne Bell Smackers Bath and Body Starburst Collection", "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz", "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz", "Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce", \n Whether the user will like the target movie "Naturelle Hypo-Allergenic Styling Gel"?', 'output': 'Yes.'}
Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".
User Preference:"Citre Shine Moisture Burst Shampoo - 16 fl oz", "Bonne Bell Smackers Bath and Body Starburst Collection", "Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 o

1